# Coursework for BEMM461 Project 

## Introduction
This dashboard analyzes Airbnb listings in London to reveal optimal pricing strategies that balance revenue growth with high customer satisfaction. By integrating financial performance metrics with geographic data and customer feedback, the dashboard provides actionable insights for inventory management, targeted pricing, and customer experience improvements. The key is to leverage high-demand zones while strategically managing pricing in less central locations to maintain customer satisfaction and overall profitability.

## Table of Links
### Table
| Description | Link |
| -- | -- |
| Reflective blog | https://ele.exeter.ac.uk/mod/oublog/view.php?id=3234123 |
| Dashboard | Insert Link here |
| Chosen Dataset | https://zenodo.org/records/4446043#.Y9Y9ENJBwUE /  https://zenodo.org/records/4446043/files/london_weekdays.csv /  https://zenodo.org/records/4446043/files/london_weekends.csv|
                    
## Table of Contents
1. Executive Summary
2. Project Dashboard
3. Background to the Project
4. Articulation of Decision Making Process
5. Review of Analytics Methods Chosen
6. Review of Available Tools
7. Review of Chosen Datasets 
8. Visualisation of Data with Accompanying Code
9. Reflective Evaluation
10. Conclusion


## 1. Executive Summary

This report summarizes a BEMM461 project that built an interactive dashboard to seek for insights for Airbnb pricing strategy improvement in London. The project showcases the development of analytical and visualization skills through the creation of this key tool. The report explains the project's background, the reasoning behind the data analysis choices, and a critical assessment of the methods and software used (the Dash Python framework). Data transparency is ensured by providing access to the datasets employed. The Python code is included for review and understanding. A supporting blog post details the project's progression, challenges overcome, and lessons learned.

The dashboard itself presents a comprehensive overview of nearly 10,000 London Airbnb listings. Key metrics, including average price, customer satisfaction ratings, and distance from the city center, are displayed. The visualizations highlight the relationship between price, location, and guest satisfaction. Central London listings command higher prices, while a slight negative correlation exists between distance and satisfaction levels. The analysis also shows the significant revenue contribution of entire apartment/home listings.

In short, this project uses data visualization to offer insights into optimizing pricing and improving the customer experience for Airbnb in London. The findings offer actionable recommendations for pricing strategies and resource allocation. The entire process, from data selection to final conclusions, is documented and supported by relevant sources.


## 2. Project Dashboard

The Airbnb London dashboard is designed with a “Monitor, Analyze, Drill to Detail” approach to facilitate clear and efficient data interpretation for decision-making. Central to its design are three core visualization theories. Firstly, Edward Tufte’s principles of visualization emphasize maximizing the data-ink ratio. This is achieved by prioritizing clear data representation and minimizing unnecessary visual elements, ensuring that the focus remains squarely on the data’s message (Tufte, 2001). The charts employ straightforward designs, clear labels, and a restrained color palette to avoid clutter and enhance readability.

Secondly, the dashboard adheres to Shneiderman’s Mantra of interactive design. This involves a three-stage exploration: an initial overview presenting key performance indicators (KPIs); interactive filtering mechanisms (dropdowns and range sliders) allowing users to zoom in on specific data segments; and finally, detailed visualizations providing deeper insights upon user interaction (Shneiderman, 1996). Crucially, all graphs update dynamically and consistently based on filter selections, ensuring a seamless and intuitive exploration process.

Finally, the design incorporates Cognitive Load Theory to minimize mental effort. This is accomplished through logical information structuring, clear visual hierarchies, and a restrained color palette to avoid visual overload (Sweller, 1988). The selection of appropriate chart types—pie charts, bar charts, heatmaps, scatter plots, and line charts—further enhances data comprehension at a glance. By adhering to these three visualization theories, the dashboard aims to provide a user-friendly and insightful experience, empowering managers and consultants to make informed decisions based on the data presented.

In [1]:
import pandas as pd
import dash_bootstrap_components as dbc
import plotly.express as px
from dash import Dash, dcc, html, Input, Output

#Loading and prepare data
weekdays_df = pd.read_csv('https://zenodo.org/records/4446043/files/london_weekdays.csv')
weekdays_df['Day Type'] = 'Weekday'
weekends_df = pd.read_csv('https://zenodo.org/records/4446043/files/london_weekends.csv')
weekends_df ['Day Type']= 'Weekend'

df = pd.concat([weekdays_df, weekends_df], ignore_index= True)
df.rename(columns={'realSum': 'Price',
                  'guest_satisfaction_overall': 'Satisfaction',
                  'cleanliness_rating': 'Cleanliness',
                  'room_type': 'Room Type',
                  'host_is_superhost': 'Superhost status',
                  'dist': 'City Distance'}, inplace = True)

#Initializing dash app:
app = Dash(__name__, external_stylesheets=[dbc.themes.BOOTSTRAP])
server = app.server

In [2]:
print (df.columns)
print(df.head())
print(df.tail())
print(df['Day Type'].value_counts())

Index(['Unnamed: 0', 'Price', 'Room Type', 'room_shared', 'room_private',
       'person_capacity', 'Superhost status', 'multi', 'biz', 'Cleanliness',
       'Satisfaction', 'bedrooms', 'City Distance', 'metro_dist', 'attr_index',
       'attr_index_norm', 'rest_index', 'rest_index_norm', 'lng', 'lat',
       'Day Type'],
      dtype='object')
   Unnamed: 0       Price        Room Type  room_shared  room_private  \
0           0  570.098074  Entire home/apt        False         False   
1           1  297.984430     Private room        False          True   
2           2  336.790611     Private room        False          True   
3           3  226.722171     Private room        False          True   
4           4  256.355982     Private room        False          True   

   person_capacity  Superhost status  multi  biz  Cleanliness  ...  bedrooms  \
0              2.0             False      0    0         10.0  ...         1   
1              2.0              True      1    0       

In [ ]:
#App layout
app.layout = dbc.Container([
    
    #Title:
    html.Div([
        html.H1('Airbnb London - Distance and Pricing Insights Dashboard ',
                className = 'text-center mb4',
                style ={'color': 'white', 'background-color':'#20283E', 'padding':'10px','border-radius': '5px'})]),    
    
    #Overview:
    dbc.Row ([
        dbc.Col(dbc.Card([
            dbc.CardBody([
            html.H5('Total Listings', className="text-center", style={'font-weight': 'bold'}),
            html.H2(id='total_listings', className="text-center", style={'color': '#AC3E31', 'font-weight': 'bold'})
            ])
        ], className ='shadow - sm'), width = 3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Average Price", className="text-center", style={'font-weight': 'bold'}),
                html.H2(id="average_price", className="text-center", style={'color': '#AC3E31', 'font-weight': 'bold'})
            ])
        ], className="shadow-sm"), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Average Satisfaction", className="text-center", style={'font-weight': 'bold'}),
                html.H2(id="avg_satisfaction", className="text-center", style={'color': '#AC3E31', 'font-weight': 'bold'})
            ])
        ], className="shadow-sm"), width=3),

        dbc.Col(dbc.Card([
            dbc.CardBody([
                html.H5("Avg Distance to City Center", className="text-center", style={'font-weight': 'bold'}),
                html.H2(id="avg_distance", className="text-center", style={'color': '#AC3E31', 'font-weight': 'bold'})
            ])
        ], className="shadow-sm"), width=3)
    ], className="mb-4", style={'background-color': '#DADADA', 'padding': '20px', 'border-radius': '5px'}),
    
    #Filter:
    dbc.Row([
        dbc.Col(html.Div([
            html.H5('Room Type', style = {'font-weight': 'bold', 'color' : 'white'}),
            dcc.Dropdown(id='room_type_filter', options=[{'label': room, 'value': room} for room in df['Room Type'].unique()], placeholder="Select Room Type")
            ]), width=3),
        
        dbc.Col(html.Div([
            html.H5("Price Range", style={'font-weight': 'bold', 'color' : 'white'}),
            dcc.RangeSlider(id='price_filter',
                            min=0, max=2500, 
                            step=50,
                            marks={0: '0', 500: '500', 1000: '1000', 1500: '1500', 2000: '2000', 2500: 'High'},
                            value=[0, 2500])
        ]), width=3),
        
        dbc.Col(html.Div([
            html.H5("Distance Range", style={'font-weight': 'bold', 'color' : 'white'}),
            dcc.RangeSlider(id='distance_filter',
                            min=0, max=20,
                            step=1,
                            marks={0: 'Near', 5: '5 km', 10: '10 km', 15: '15 km', 20: '20'},
                            value=[0, 20])
            ]), width=3),

        dbc.Col(html.Div([
            html.H5("Satisfaction Tiers", style={'font-weight': 'bold', 'color' : 'white'}),
            dcc.RangeSlider(id='satisfaction_filter',
                            min=0, max=100, step=1,
                            marks={0: 'Low',75: 'Medium', 100: 'High'},
                            value=[0, 100])
            ]), width=3),

        dbc.Col(html.Div([
            html.H5("Day Type", style={'font-weight': 'bold', 'color' : 'white'}),
            dcc.Dropdown(id='day_type_filter',
                         options=[{'label': 'Weekday', 'value': 'Weekday'},
                                  {'label': 'Weekend', 'value': 'Weekend'}],
                         placeholder="Select Day Type")
        ]), width=3),
    ], className="mb-4", style={'background-color': '#20283E', 'padding': '20px', 'border-radius': '5px'}),

# Graphs Section
    dbc.Row([
        dbc.Col(dcc.Graph(id='room_type_pie_chart'), width=6),
        dbc.Col(dcc.Graph(id='revenue_bar_chart'), width=6)
    ]),
    dbc.Row([
        dbc.Col(dcc.Graph(id='price_heat_map'), width=8),
        dbc.Col(dcc.Graph(id='price_vs_city_distance'), width=4)
    ]),
    dbc.Row([dbc.Col(dcc.Graph(id='distance_vs_satisfaction'), width=12)])
], fluid=True)
#App call back:
@app.callback(
    [
        Output('total_listings', 'children'),
        Output('average_price', 'children'),
        Output('avg_satisfaction', 'children'),
        Output('avg_distance', 'children'),
        Output('room_type_pie_chart', 'figure'),
        Output('revenue_bar_chart', 'figure'),
        Output('price_heat_map', 'figure'),
        Output('price_vs_city_distance', 'figure'),
        Output('distance_vs_satisfaction', 'figure')
    ],
    [
        Input('room_type_filter', 'value'),
        Input('price_filter', 'value'),
        Input('distance_filter', 'value'),
        Input('satisfaction_filter', 'value'),
        Input('day_type_filter', 'value')
    ]
)
def update_dashboard(room_type, price_range, distance_range, satisfaction_range, day_type):
    # Filter data based on input:
    filtered_df = df.copy()

    if room_type:
        filtered_df = filtered_df[filtered_df['Room Type'] == room_type]

    filtered_df = filtered_df[
        (filtered_df['Price'] >= price_range[0]) &
        (filtered_df['Price'] <= price_range[1])
    ]
    filtered_df = filtered_df[
        (filtered_df['City Distance'] >= distance_range[0]) &
        (filtered_df['City Distance'] <= distance_range[1])
    ]
    filtered_df = filtered_df[
        (filtered_df['Satisfaction'] >= satisfaction_range[0]) &
        (filtered_df['Satisfaction'] <= satisfaction_range[1])
    ]
    if day_type:
        filtered_df = filtered_df[filtered_df['Day Type'] == day_type]

    # Overview Metrics
    total_listings = len(filtered_df)
    average_price = f"{filtered_df['Price'].mean():.2f} EUR"
    avg_satisfaction = f"{filtered_df['Satisfaction'].mean():.2f}"
    avg_distance = f"{filtered_df['City Distance'].mean():.2f} km"

    # Graphs
    room_type_pie = px.pie(filtered_df, names='Room Type', title="Room Type Distribution")
    
    revenue_bar = px.bar(filtered_df.groupby('Room Type')['Price'].sum().reset_index(),
                         x='Room Type', y='Price', title="Revenue by Room Type")
    revenue_bar.update_layout(yaxis_title="Total Revenue (EUR)")
    
    heat_map = px.density_mapbox(filtered_df, lat='lat', lon='lng', z='Price', radius=10,
                                 center=dict(lat=df['lat'].mean(), lon=df['lng'].mean()),
                                 mapbox_style="carto-positron", title="Geographic Price Distribution")
    
    price_vs_distance = px.line(filtered_df.groupby('City Distance')['Price'].mean().reset_index(),
                                x='City Distance', y='Price', title="Average Price vs City Distance")
    
    distance_vs_satisfaction = px.scatter(filtered_df, x='City Distance', y='Satisfaction',
                                          trendline='ols', title="Distance vs Guest Satisfaction",
                                          labels={'City Distance': 'Distance (km)', 'Satisfaction': 'Satisfaction'},
                                          color_discrete_sequence=['#488A99'])
    distance_vs_satisfaction.update_traces(
        line=dict(color='#AC3E31'),  # Change trendline color to red
        selector=dict(mode='lines'))

    return total_listings, average_price, avg_satisfaction, avg_distance, room_type_pie, revenue_bar, heat_map, price_vs_distance, distance_vs_satisfaction

In [4]:
# Run the App
if __name__ == "__main__":
    app.run_server(debug=True, port = 8051)

## 3. Background to the Project

The Airbnb Insights Dashboard is developed to address the complex challenges of optimizing pricing strategies, improving customer satisfaction, and driving revenue growth for Airbnb in London. By leveraging two datasets—london_weekdays.csv and london_weekends.csv—this project evaluates key metrics, including pricing trends, guest satisfaction, room type distribution, and geographical price variations. The dashboard aims to equip decision-makers, such as Airbnb’s C-level team, with actionable insights to balance profitability and customer experience, consider distance to city center as a strategic factor.

Furthermore, a study by Zervas et al. (2017) estimated the impact of Airbnb on the traditional hotel industry. The researchers found that Airbnb's disruptive presence in the market has led to a significant decline in hotel revenue, particularly in the lower-end segment. This highlights the intense competition Airbnb faces from traditional hospitality providers and the need to maintain a strong value proposition for both hosts and guests. Therefore, it can be seen that Airbnb’s business operates in a competitive and dynamic environment where pricing and customer satisfaction are crucial factors for sustained growth. 

Therefore, in order to improve the platform's competitive advantage the dashboard aim to provide insigh for:

    Portfolio Optimization: By analyzing room type distribution and revenue contributions, Airbnb can align its offerings with market demand and profitability.

    Revenue Maximization: The geographic price distribution and city distance analysis highlight high-demand areas and pricing trends, providing opportunities to implement dynamic pricing and investment strategies.

    Customer-Centric Growth: Insights into guest satisfaction relative to distance can guide improvements in amenities, pricing, and services to boost customer experience and loyalty.

The dataset being used provide detailed information on pricing, guest satisfaction scores, and geographical characteristics of listings, enabling analysis across key dimensions such as room type, proximity to the city center, and temporal patterns (weekdays vs. weekends). These insights are visualized through interactive graphs and metrics to enable managers to make data-driven decisions efficiently.

## 4. Articulation of Decision Making Process
The main focus of the Airbnb Insights Dashboard project was to develop a tool that supports strategic decision-making for inventory optimization, revenue growth, and customer satisfaction enhancement. This required iterative adjustments and careful consideration of data visualization principles, as reflected in the weekly blog updates.

As noted in Week 1: Dataset Selection, the decision to use the Airbnb datasets (london_weekdays.csv and london_weekends.csv) stemmed from their comprehensive nature, providing detailed information on pricing, satisfaction scores, room types, and location data. The blog highlighted the rationale behind selecting these datasets due to their versatility and pre-existing familiarity, allowing for effective visualization while maintaining flexibility in adjusting the scope as needed [[Week 1 Blog](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=45382)].

During Week 2: Dashboard Visualization, the project direction shifted towards building a visually compelling dashboard with interactivity at its core. Theories such as Edward Tufte’s data-ink ratio and Shneiderman’s Mantra were adopted to guide the dashboard design. This decision focused on creating visuals that maximize data clarity while supporting exploration and filtering. For instance, pie charts were chosen for room type distribution to emphasize portfolio composition, and heat maps were incorporated to visualize high-demand zones in the London market [Week 2 Blog](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=45754).

In Week 3: Audience and Insight, a crucial decision was made to refine the target audience and focus the dashboard on insights relevant to Airbnb’s C-level executives. This required aligning the graphs with key performance indicators such as portfolio composition, revenue by room type, and customer satisfaction. The blog emphasized the shift from general data visualization to actionable insights that directly inform business strategy [Week 3 Blog](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=46099).

By Week 4: Graph Adjustment, the dashboard’s storytelling elements became more structured, integrating a narrative centered on inventory optimization, revenue growth, and customer-centric improvements. Adjustments included narrowing the price range to eliminate outliers and adding a “Weekdays vs. Weekends” filter to distinguish temporal trends in demand. This aligns with the reflective blog’s acknowledgment of the need for enhanced clarity and targeted insights for decision-makers [Week 4 Blog](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=46212).

In Week 5: Visualizing and Finalizing, the focus shifted to implementation and bug resolution. The web-based version was finalized after extensive testing to ensure filters, interactivity, and visualizations functioned as intended. Challenges such as callback errors and uneven datasets were resolved through research and debugging, demonstrating the iterative nature of the decision-making process. Throughout the report cond ucting progress, the author decided to adopt a web-based dashboard to optimize it's delivery. Further understanding relate to the tools (HEROKU) to make an app for the dashboard was required [Week 5 Blog](https://ele.exeter.ac.uk/mod/oublog/viewpost.php?post=46361).

## 5. Review of Analytics Methods Chosen
The Airbnb Insights Dashboard leverages a combination of analytics methods and visualization techniques to derive actionable insights from its dataset. Visualization is central to this process, as it transforms raw data into intuitive interactive formats that are easily understood.

The dashboard incorporates descriptive analytics, which provides an overview of key metrics such as total listings, average price, guest satisfaction scores, and average distance to the city center. These metrics are presented through visually distinct cards, offering a quick snapshot of performance indicators. Descriptive analytics plays a foundational role in understanding “what has happened” within the data, forming the basis for further exploration (Camm et al., 2020).

Exploratory Data Analysis (EDA) is utilized to uncover patterns and relationships within the data through visualizations such as pie charts, bar charts, scatter plots, heatmaps, and line charts. Pie charts highlight room type distributions, enabling Airbnb to evaluate its portfolio composition, while bar charts visualize revenue contributions by room type, emphasizing the profitability of specific categories. Heatmaps, on the other hand, provide geographic price distributions, revealing high-demand zones and areas with untapped potential. These visualizations facilitate the identification of trends and anomalies, aiding Airbnb in strategic decision-making (Tukey, 1977).

To explore relationships between variables, correlation analysis is applied through scatter plots with trendlines, examining the relationship between city distance and guest satisfaction. These visualizations help identify weak correlations, suggesting that guest satisfaction is influenced more by factors other than proximity to the city center. Interactive filters for room type, price range, satisfaction tiers, and day type allow users to dynamically segment the data, aligning with diagnostic analytics, which seeks to explain why certain patterns occur by breaking down the data into meaningful subsets (Laursen & Thorlund, 2016).

Moreover, The dashboard also supports predictive analytics indirectly by revealing trends, such as pricing patterns relative to proximity to the city center. For instance, line charts help visualize how price fluctuations are tied to geographic factors, enabling Airbnb to forecast pricing strategies for maximizing occupancy and revenue. Finally, the dashboard design adheres to Edward Tufte’s principles of maximizing the data-to-ink ratio and Shneiderman’s mantra of “Overview first, zoom and filter, then details on demand,” reducing cognitive load and ensuring ease of use (Tufte, 2001; Shneiderman, 1996)

## 6. Review of Available Tools

Power BI is a widely adopted business intelligence tool with an intuitive interface and built-in visualizations, making it ideal for creating interactive dashboards. Its drill-down and filtering capabilities, along with machine learning integration, would streamline predictive analytics. However, Power BI’s free version limits data capacity and sharing, while advanced features require proficiency in DAX (Microsoft, 2024).

Tableau, another leading visualization tool, excels at creating highly interactive and visually appealing dashboards. Its drag-and-drop interface and built-in statistical features make it effective for exploring Airbnb datasets. However, Tableau’s steep learning curve and licensing costs may pose challenges for non-technical users or small-scale projects (Murray, 2016).

## 7. Review of Chosen Datasets 

The datasets london_weekdays and london_weekends were highly suitable for this project as they offer a detailed view of Airbnb listings in London. Covering variables like pricing, guest satisfaction, cleanliness, room type, and geographic distance, these datasets provide the foundation for exploring temporal trends, such as differences in demand between weekdays and weekends. This alignment with the project’s goals made them ideal for generating insights into pricing strategies, portfolio composition, and customer satisfaction

The datasets enabled a variety of visualizations crucial for the dashboard, including pie charts (room type distribution), bar charts (revenue by room type), and heatmaps (geographic price distribution). Key attributes like latitude and longitude facilitated geospatial analysis, while customer-centric data such as satisfaction and cleanliness helped evaluate the operational impact on guest perceptions. This versatility was noted in Blog Post 1, where the dataset’s comprehensiveness was seen as a “safe choice” due to its flexibility for various analyses.

However, challenges arose during data preprocessing. The datasets had inconsistencies in row counts and missing values, requiring cleaning to ensure accurate integration. For example, outliers in the price variable were capped at 2500 EUR, covering 97% of the data, as detailed in Blog Post 5. Despite these obstacles, the datasets ultimately supported the project’s analytical objectives.

## 8. Visualisation of Data with Accompanying Code

Strategic visualizations for the Airbnb Insights Dashboard and their significance in addressing the business objectives were selected based on their ability to provide actionable insights for Airbnb’s decision-making. Moreover, the storytelling flow of the dashboard is structured using the principles of progressive disclosure and visual hierarchy, ensuring that insights are communicated in a logical, user-centric manner (Shneiderman, 1996; (Tufte, 2001)). The goal of the cosen order is to enable the audience (Airbnb’s decision-makers) to gain high-level insights first, then progressively drill down into specific details based on their interests and business priorities. Further justification of the dashboard insight and order will be shown below.

| Visualization | Justification|
| -- | -- |
| Room Type Distribution (Pie Chart) | Provides an inventory snapshot, emphasizing portfolio composition and alignment with customer demand. |
| Room Type Distribution (Pie Chart) | : HShifts focus to profitability, helping prioritize investments and adjust strategies for underperforming categories.|
| Geographic Price Distribution (Heatmap) | Highlights high-demand zones, aiding in optimizing pricing strategies and identifying expansion opportunities. |
| Distance vs. Guest Satisfaction (Scatter Plot) | Explores how location impacts satisfaction, offering insights into customer perception and willingness to pay. |
| Average Price vs. City Distance (Line Chart) | : Helps balance pricing strategies with geographical trends to maximize revenue and occupancy|

In [6]:
#Accompanying code for section 8
room_type_pie = px.pie(filtered_df, names='Room Type', title="Room Type Distribution")
    
revenue_bar = px.bar(filtered_df.groupby('Room Type')['Price'].sum().reset_index(),
                         x='Room Type', y='Price', title="Revenue by Room Type")
revenue_bar.update_layout(yaxis_title="Total Revenue (EUR)")
    
heat_map = px.density_mapbox(filtered_df, lat='lat', lon='lng', z='Price', radius=10,
                                 center=dict(lat=df['lat'].mean(), lon=df['lng'].mean()),
                                 mapbox_style="carto-positron", title="Geographic Price Distribution")
    
price_vs_distance = px.line(filtered_df.groupby('City Distance')['Price'].mean().reset_index(),
                                x='City Distance', y='Price', title="Average Price vs City Distance")
    
distance_vs_satisfaction = px.scatter(filtered_df, x='City Distance', y='Satisfaction',
                                          trendline='ols', title="Distance vs Guest Satisfaction",
                                          labels={'City Distance': 'Distance (km)', 'Satisfaction': 'Satisfaction'},
                                          color_discrete_sequence=['#488A99'])
distance_vs_satisfaction.update_traces(
    line=dict(color='#AC3E31'),  # Change trendline color to red
    selector=dict(mode='lines'))

NameError: name 'filtered_df' is not defined

## 9. Reflective Evaluation

The project process was both challenging and rewarding, requiring a combination of technical, analytical, and creative skills. One significant challenge was handling the interactivity of the dashboard. Implementing dynamic filters for room type, price range, satisfaction tiers, and distance, while ensuring all visualizations updated seamlessly, introduced debugging complexities. Callback errors in Dash and dataset inconsistencies tested both the problem-solving and technical skills of the creator. However, iterative testing and thorough exploration of Dash’s capabilities enabled the creator to overcome these hurdles, improving expertise in Python and dashboard development.

Visualization was central to the project’s success, adhering to Tufte’s principles of maximizing the data-to-ink ratio and Shneiderman’s mantra of “Overview first, zoom and filter, then details on demand.” The selected visualizations—such as pie charts for portfolio composition, scatter plots for correlation analysis, and heatmaps for spatial trends—were designed to convey insights effectively while maintaining clarity.

The experience underscored the importance of iterative development and feedback. Challenges such as dataset preprocessing and callback errors reinforced the value of careful planning and modular design. Lessons learned include better time allocation for data cleaning and leveraging alternative tools for prototyping. Overall, the project strengthened the creator’s technical, analytical, and storytelling skills, offering valuable insights for future visualization and analytics projects.

## 10. Conclusion

The Airbnb Insights Dashboard project successfully achieved its objectives by delivering a comprehensive, interactive visualization tool that provides actionable insights for decision-making. The dashboard was designed with user-centric principles, adhering to Tufte’s data visualization guidelines and Shneiderman’s interactive design mantra, ensuring clarity, usability, and relevance.

Furthermore, the project demonstrated the importance of data preprocessing, including handling inconsistencies and outliers, which formed the foundation for accurate and meaningful visualizations. Key visualizations, such as scatter plots for correlation analysis and heatmaps for geographic trends, facilitated strategic insights into pricing optimization and customer satisfaction. While the dynamic filters incorporated into the dashboard empowered users to explore and segment data interactively, enhancing its practical utility for Airbnb’s strategic objectives, Challenges faced during the project, particularly debugging callback errors and managing large datasets, reinforced the value of iterative development, modular design, and feedback.

In conclusion, the project highlighted the transformative role of data visualization in bridging raw data with actionable insights. It not only strengthened the creator’s technical and analytical skills but also emphasized the value of storytelling in driving informed decisions for business growth and customer satisfaction.


## References

Tufte, E. R. (2001). The visual display of quantitative information. Cheshire, CT: Graphics Press.

Shneiderman, B. (1996). The eyes have it: A task by data type taxonomy for information visualizations. Proceedings 1996 IEEE Symposium on Visual Languages, 336–343. https://doi.org/10.1145/223904.223910.

Sweller, J. (1988). Cognitive load during problem solving: Effects on learning. Cognitive Science, 12(2), 257–285. https://pmc.ncbi.nlm.nih.gov/articles/PMC10361235/.

Camm, J. D., Cochran, J. J., Fry, M. J., & Ohlmann, J. W. (2020). Business analytics. Cengage Learning.

Tukey, J. W. (1977). Exploratory data analysis. Pearson.

Laursen, G., & Thorlund, J. (2016). Business analytics for managers: Taking business intelligence beyond reporting. Wiley.

Microsoft. (2024). Power BI overview. Retrieved from https://powerbi.microsoft.com.

Murray, D. (2016). Tableau your data!. Wiley.

Google. (2024). Data Studio features. Retrieved from https://datastudio.google.com.

Airbnb London Weekday and Weekend Dataset. (2024). Retrieved from https://zenodo.org/records/4446043#.Y9Y9ENJBwUE.

Egger, R., & Yu, J. (2021). Data-driven insights for tourism and Airbnb analytics. Tourism Management, 85, 104319. Retrieved from https://www.sciencedirect.com/science/article/pii/S0261517721000388?via%3Dihub.